Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [1]:
NAME = "Artem Burtsev"
COLLABORATORS = "Me and only"

---

# Integrable singularities

Compue the integral

$$
I = \int_{0}^{1}\frac{e^x}{\sqrt{x(1-x)}}dx.
$$

using the trapezoid formula. Note that the integrand diverges at the integration limits and the singularities are integrable (the integral itself exists). So, you will need to subtract these singularities.

The following integrals might be useful:

$$
\int_0^1 \frac{1}{\sqrt{x (1-x)}} \, dx=\pi,\quad \int_0^1 \frac{x}{\sqrt{x (1-x)}} \, dx=\pi/2.
$$

Transform the integral to subtract the singularities. Write out the formulas in the cell below (5pt, manual grading).

$$
I = \int_{0}^{1}\dfrac{e^x}{\sqrt{x-x^2}}dx = \int_0^1 \dfrac{e^x - 1}{\sqrt{x-x^2}}dx + \int_0^1 \dfrac{1}{\sqrt{x-x^2}}dx = \int_0^1 \dfrac{e^x - 1}{\sqrt{x-x^2}}dx + \pi
$$

Thus, $\lim\limits_{x\to 0} \dfrac{e^x - 1}{\sqrt{x-x^2}} = 0$. At the same time, 

$$
I = \int_{0}^{1}\dfrac{e^x}{\sqrt{x-x^2}}dx = \int_0^1 \dfrac{e^x - e}{\sqrt{x-x^2}}dx + e\int_0^1 \dfrac{1}{\sqrt{x-x^2}}dx = \int_0^1 \dfrac{e^x - e}{\sqrt{x-x^2}}dx + \pi e
$$

Where $\lim\limits_{x\to 1} \dfrac{e^x - e}{\sqrt{x-x^2}} = 0$

Overall, $I = \int\limits_{0}^{1}\dfrac{e^x}{\sqrt{x-x^2}}dx = \int\limits_0^{0.5} \dfrac{e^x - 1}{\sqrt{x-x^2}}dx + \int\limits_{0.5}^1 \dfrac{e^x - e}{\sqrt{x-x^2}}dx + \dfrac{\pi}{2}(1 + e)$

Write a function which computes the value of the integrals using the trapezoid rule.

In [2]:
import numpy as np
from numba import prange, njit

@njit(parallel=True)
def integ(npts=10):
    """Compute the value of the integral above.
    
    Subtract the singularities and use the trapezoid rule. 
    
    Parameters
    ----------
    npts : int
        The number of points for the trapezoid rule
        
    Returns
    -------
    I : float
       The computed value of the integral
    """
    
    left_limit = (lambda x: (np.exp(x) - 1) / np.sqrt(x - x**2))
    right_limit = (lambda x: (np.exp(x) - np.e) / np.sqrt(x - x**2)) 
    
    X = np.linspace(0, 1, npts + 1)
    step = 1 / npts
    
    result = 0
    for point in prange(len(X) - 1):
        if X[point] < 0.5 and X[point] != 0:
            local_sum = left_limit(X[point]) + left_limit(X[point + 1])
            result += 0.5 * step * local_sum
        elif X[point] >= 0.5 and X[point + 1] != 1:
            local_sum = right_limit(X[point]) + right_limit(X[point + 1])
            result += 0.5 * step * local_sum
            
    return result + np.pi/2 * (np.e + 1)

In [3]:
integ(100000000)

5.508429773887325

In [4]:
### this is a test to check your computed value
from numpy.testing import assert_allclose
